In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My Drive

/content/gdrive/My Drive


Model Training

In [18]:
import numpy as np
import torch # tensor library
from torch import nn # neural network library
from torch.utils.data import DataLoader, TensorDataset # libraries for data loading and combining tensors
from sklearn.model_selection import KFold
# Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).
# Each fold is then used once as a validation while the k - 1 remaining folds form the training set.
import pandas as pd


class Net(nn.Module): # inheritance of base class for NN nn.module
    # Initialize the layers
    def __init__(self):
        super().__init__() # for calling constructor of nn.module
        self.linear1 = nn.Linear(8, 30) # hidden layer, input = 8, output = 30
        self.act1 = nn.ReLU() # Activation function
        self.linear2 = nn.Linear(30, 1) # output layer, input = 30, output = 1
    
    # Perform the computation
    def forward(self, x):
        x = self.linear1(x) 
        x = self.act1(x)
        x = self.linear2(x)
        return x


if __name__ == '__main__':

    # Configuration options
    k_folds = 5
    num_epochs = 15
    loss_funct = nn.MSELoss()
  
    # For fold results
    results = {}
  
    # Set fixed random number seed
    torch.manual_seed(42)


    Xtr=np.loadtxt("TrainData.csv") 
    Ytr=np.loadtxt("TrainLabels.csv")

    # conversion from numpy to tensors
    Xdat = torch.from_numpy(Xtr).type(torch.float)
    Ydat = torch.from_numpy(Ytr).type(torch.float).unsqueeze(-1) # -1 to add dimension to match dimension of x

    dataset = TensorDataset(Xdat, Ydat) # combining two tensors in one dataset

    kfold = KFold(n_splits=k_folds, shuffle=True) # load kfold function

    for fold, (train, val) in enumerate(kfold.split(dataset)):   # k-fold validation, iterating through dataset and spliting into training and testing sets
    
      print(f'\nFOLD No. {fold}')
      print("---------------\n\n")
    
      # Sample elements randomly from a given list of idexes with no replacement.
      train_sample = torch.utils.data.SubsetRandomSampler(train)
      val_sample = torch.utils.data.SubsetRandomSampler(val)
    
      # data loaders for training and testing
      trainloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=train_sample)
      valloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=val_sample)
    
      model = Net()
    
      # optimizer
      optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    
      # training for number of epochs
      for epoch in range(0, num_epochs):

        # Print epoch
        print(f'epoch no. {epoch+1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
        
          # Get inputs
          input, target = data
        
          # reset gradients to zero so it won't accumalate nxt time
          optimizer.zero_grad()
        
          # Perform forward pass
          output = model(input)
        
          # Compute RMSE
          loss = torch.sqrt(loss_funct(output, target))
        
          # Perform backward pass
          loss.backward()
        
          # Perform optimization
          optimizer.step()
        
          # calculate loss
          current_loss += loss.item()
          if (i%20 == 19):
            print('RMSE Loss: ', current_loss/ 20)
            print("\n")
            current_loss = 0.0

      # Saving model
      file = 'my_Model.pth'
      torch.save(model.state_dict(), file)



FOLD No. 0
---------------


epoch no. 1
RMSE Loss:  49.15206604003906


epoch no. 2
RMSE Loss:  17.06835255622864


epoch no. 3
RMSE Loss:  12.120721864700318


epoch no. 4
RMSE Loss:  11.831024789810181


epoch no. 5
RMSE Loss:  11.246107172966003


epoch no. 6
RMSE Loss:  10.88050193786621


epoch no. 7
RMSE Loss:  10.431378269195557


epoch no. 8
RMSE Loss:  10.877173447608948


epoch no. 9
RMSE Loss:  9.565579557418824


epoch no. 10
RMSE Loss:  8.667272639274596


epoch no. 11
RMSE Loss:  8.93996000289917


epoch no. 12
RMSE Loss:  7.889281177520752


epoch no. 13
RMSE Loss:  7.626028275489807


epoch no. 14
RMSE Loss:  7.53884654045105


epoch no. 15
RMSE Loss:  7.487076711654663



FOLD No. 1
---------------


epoch no. 1
RMSE Loss:  40.05458288192749


epoch no. 2
RMSE Loss:  12.18209891319275


epoch no. 3
RMSE Loss:  10.307553172111511


epoch no. 4
RMSE Loss:  10.89247887134552


epoch no. 5
RMSE Loss:  9.93261477947235


epoch no. 6
RMSE Loss:  11.129143834114075


epoch 

**Model Prediction**


In [19]:
# Loading Data
Yts=np.loadtxt("TestData.csv") 
Ytsdat = torch.from_numpy(Yts).type(torch.float)

# Loading the model
mod = Net()
predict = mod(Ytsdat)
mod.load_state_dict(torch.load(file))
mod.eval()


p = predict.detach().numpy()
p_df = pd.DataFrame(p)
p_df.to_csv('20I-2003_predictions.csv')

In [20]:
df = pd.read_csv (r'20I-2003_predictions.csv')
print(df)

     Unnamed: 0          0
0             0  33.415604
1             1  45.237186
2             2  47.556915
3             3  47.575443
4             4  59.302307
..          ...        ...
201         201  40.068672
202         202  26.839231
203         203  23.195105
204         204  28.061060
205         205  34.845627

[206 rows x 2 columns]
